# 0. Imports

In [32]:
# pip freeze > requirements.txt

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import os


# 1. Training Data

In [ ]:
# -------------------------
# 1. Preparar dados
# -------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

if os.path.exists("./00_TrainingData"):
    trainset = torchvision.datasets.CIFAR10(root='./00_TrainingData', train=True,
                                            download=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                            shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./00_TrainingData', train=False,
                                        download=False, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                            shuffle=False)
else:
    trainset = torchvision.datasets.CIFAR10(root='./00_TrainingData', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                            shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./00_TrainingData', train=False,
                                        download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                            shuffle=False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 2. CNN Architecture

In [14]:
# -------------------------
# 2. Definir CNN
# -------------------------
class AnimalCNN(nn.Module):
    def __init__(self):
        super(AnimalCNN, self).__init__()
        
        # Bloco 1
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout_conv = nn.Dropout(0.25)
        
        # Bloco 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.dropout_conv2 = nn.Dropout(0.25)
        
        # Fully connected
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # 10 classes do CIFAR-10

    def forward(self, x):
        # Bloco 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout_conv(x)
        
        # Bloco 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.dropout_conv2(x)
        
        # Flatten
        x = x.view(-1, 128 * 8 * 8)
        
        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        
        return x

# -------------------------
# 3. Treinar modelo
# -------------------------

net = AnimalCNN().to(device)

# 3. CNN Training

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

# -------------------------
# 4. Treinamento
# -------------------------
num_epochs = 50

for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    scheduler.step()
    train_loss = running_loss / len(trainloader.dataset)
    train_acc = 100. * correct / total
    
    # Avaliação no teste
    net.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    test_acc = 100. * correct_test / total_test
    
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")


Epoch 1/50 | Loss: 1.8998 | Train Acc: 33.78% | Test Acc: 46.84%
Epoch 2/50 | Loss: 1.3290 | Train Acc: 51.72% | Test Acc: 58.52%
Epoch 3/50 | Loss: 1.1228 | Train Acc: 60.07% | Test Acc: 66.48%
Epoch 4/50 | Loss: 0.9958 | Train Acc: 64.67% | Test Acc: 67.39%
Epoch 5/50 | Loss: 0.9107 | Train Acc: 68.10% | Test Acc: 73.50%
Epoch 6/50 | Loss: 0.8410 | Train Acc: 70.71% | Test Acc: 74.80%
Epoch 7/50 | Loss: 0.7887 | Train Acc: 72.59% | Test Acc: 75.76%
Epoch 8/50 | Loss: 0.7507 | Train Acc: 73.74% | Test Acc: 76.22%
Epoch 9/50 | Loss: 0.7167 | Train Acc: 74.98% | Test Acc: 78.07%
Epoch 10/50 | Loss: 0.6760 | Train Acc: 76.39% | Test Acc: 78.55%
Epoch 11/50 | Loss: 0.6402 | Train Acc: 77.66% | Test Acc: 78.95%
Epoch 12/50 | Loss: 0.6131 | Train Acc: 78.76% | Test Acc: 80.27%
Epoch 13/50 | Loss: 0.5909 | Train Acc: 79.57% | Test Acc: 78.73%
Epoch 14/50 | Loss: 0.5701 | Train Acc: 80.09% | Test Acc: 79.39%
Epoch 15/50 | Loss: 0.5404 | Train Acc: 81.09% | Test Acc: 79.41%
Epoch 16/50 | Loss:

# 4. Testing with new data

In [19]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# Transformações iguais às usadas no treino
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # CIFAR-10 tem 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Carregar a imagem
img = Image.open("./01_TestingDataFromInternet/airplane.jpg").convert("RGB")

# Aplicar as transformações
img_tensor = transform(img)

# Adicionar dimensão de batch [1, 3, 32, 32]
img_tensor = img_tensor.unsqueeze(0)


In [20]:
net.eval()  # modo de avaliação
with torch.no_grad():
    img_tensor = img_tensor.to(device)
    outputs = net(img_tensor)
    _, predicted = torch.max(outputs, 1)

print("Classe prevista:", classes[predicted.item()])


Classe prevista: plane


# 5. Save the model

In [22]:
torch.save(net.state_dict(), 'CNN_Model.pth')
